# Data-gen notebook
Denne notebook er skabt til at holde styr på hvilke metoder Anders har brug til at generer data, og datilhørende kilder

### Indhold:
1. Simpel "<i>normal af normal</i>"
2. AR(p) processer

In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal as mn
import pandas as pd
import pandas_datareader as pdr
import statsmodels as sm
from scipy.interpolate import interp1d
from statsmodels.tsa.api import VAR

## 1. Simpel "normal af normal" data-generering
Baseret på side 51 af <b>Statistical inference of informational networks</b>.

Vi generer data baseret på
$$ X\overset{i.i.d.}{\sim}\mathcal{N}\left( 0, \sigma^2_x I \right) \\
 Y\overset{i.i.d.}{\sim}\mathcal{N}\left( X, \sigma^2_y I \right) $$

 Så er MI givet ved
 $$ I(X;\ Y) = \log_2\left( 1 + \dfrac{\sigma^2_x}{\sigma^2_y} \right) $$

In [2]:
def gen_list(n, var_x, var_y, dim=2):
    """
    Generates a set of random points according to <statistical inference of information in networks>
    on page 51. The third variable, Z, is ignored as this is only used to create training data
    for MI, not CMI.

    The covariance matrices are both a scaled identity matrix.

    :param n: Number of points realized
    :param var_x: Variance of X
    :param var_y: Variance of Y
    :param dim: Dimension of the variables X and Y
    :return:
    :returns values: a realization of the process described in the book. Dimensions are [n, X/Y, dim].
                     e.g. [20, 0, 2] gives the second element the 20th realization of X.
    :returns mi: The mutual information: I(X; Y)
    """
    values = np.zeros((n, 2, dim))  # init
    values[:, 0, :] = mn.rvs(mean=np.zeros(dim), cov=var_x * np.identity(dim), size=(n, dim)).reshape((n, dim))

    for i, x in enumerate(values[:, 0, :]):
        values[i, 1, :] = mn.rvs(mean=x, cov=var_y * np.identity(dim), size=1)

    mi = dim * np.log2(1 + var_x / var_y) / 2

    return values, mi

## 2. AR(p) processer
En AR(p) er givet ved
$$ x_k = \omega_k + \sum^{p}_{i=1}A_ix_{k-1}, \quad \omega_k \overset{i.i.d.}{\sim} \mathcal{N}\left( 0, \Sigma_\omega \right) $$

Det er åbenlyst at $E[x_k]= 0 + E[x_0]$. Hvis vi definerer $Var[x_k]=\sigma_k^2$, og $A_i(x)=A_ixA_i^\top$ så kan vi se at
$
\begin{align}
Var[x_0] &= \sigma_0^2 \\
Var[x_1] &= A_1(\sigma_0) + \Sigma_\omega \\
Var[x_2] &= A_1(\sigma_1) + A_2(\sigma_0) + \Sigma_\omega \\
&\ \ \vdots \\
Var[x_k] &= \Sigma_\omega + \sum^{p}_{i=1} A_i(\sigma_{k-i})
\end{align}
$
Dette udtryk kan udregnes løbende når vi simulerer vores AR(p) process. <i>NOTE: Der findes nogle ligninger som viser hvad variansen går imod, tror jeg. Variansen konvergerer nemlig meget hurtigt i praksis</i>

Vi kan også nemt argumenterer for at $x_k$ er fordelt efter multivariat gaussisk fordeling (da den bare er en lin.komb. af $\omega_i$)

Marginalen for et subset af $x$, $x_s$, har fordelingen:
$$ x_s \sim \mathcal{N}\left( S\mu,S\Sigma S^\top \right) $$
hvis $\mu$ og $\Sigma$ er mean og varians for $x$. Her er $S$ defineret som $s_{ij}=1$ hvis det j'te element i $x_s$ er det i'te element i $x$

Hvis vi vil undersøge MI indgangsvist kan vi tage udgangspunkt i [denne](https://www.math.nyu.edu/~kleeman/infolect7.pdf) lecture note. Det skal dog siges at det er nok noget vi selv kan udlede, eller finde en mere fast kilde på senere. På side 2 i kilden får vi givet at
$$ I(X;Y)=\dfrac{1}{2}log_2\left( \dfrac{|\Sigma_X| |\Sigma_Y|}{|\Sigma|} \right)$$
Hvor $\Sigma$ er covariancen for den jointe fordeling imellem $X$ og $Y$.

I praksis vil det sige at hvis vi tagetr $x_k^{(i)}$ til at betyde den i'te indgang i $x$ til index $k$, så får vi 
$\begin{align}
I(X_k^{(i)}; X_k^{(j)}) &= \dfrac{1}{2}log_2\left(\dfrac{ det\left(\Sigma_{ii}\right) det\left( \Sigma_{jj}\right)}{ det\left( \begin{bmatrix} \Sigma_{ii} & \Sigma_{ij} \\ \Sigma_{ji} & \Sigma_{jj} \end{bmatrix}\right) }\right) \\
&= \dfrac{1}{2}log_2\left(\dfrac{ \Sigma_{ii}\Sigma_{jj}}{\Sigma_{ii}\Sigma_{jj} -2\Sigma_{ij}}\right)
\end{align}$

In [2]:
def MI(idx1, idx2, cov):
    """Calculates the mutual information between the marginals of a multivariate gaussian distribution: I(X1; X2)

    Args:
        idx1 (int): Index of X1 in the vector it came from
        idx2 (int): Index of X2 in the vector it came from
        cov (2D array float): Covariance matrix of the vector X, which X1 and X2 came from

    Returns:
        float: Returns the mutual information of X1 and X2
    """
    top = cov[idx1, idx1] * cov[idx2, idx2]
    bot = top - 2*cov[idx1, idx2]

    return .5 * np.log2(top / bot)


def simulate_AR(coefficients, noise_cov, num_steps, x0=None):
    """Simulate the AR(p) process, with iid zero-mean gaussian additive noise.

    Args:
        coefficients (list of arrays): List of the coefficient matrices to use in the realization. Entries in list are 2D float arrays
        noise_cov (2D array float): Covariance matrix of the gaussian additive noise
        num_steps (int): Number of iterations for which the simulation should run
        x0 (1D float array, optional): The starting value of the process. If specified the derivations of the MI don't fit as of the end of september 2022. Defaults to None.
    
    Returns:
        array float: Returns the resulting array of the realization of the process
    """
    # Setup
    p = len(coefficients)
    dim = noise_cov.shape[0]
    results = np.zeros((num_steps + 1, dim))

    # initialize the initial value
    if x0 is None:
        x0 = np.zeros(p)
    results[0] = x0

    # simulate the process
    for i in range(1, num_steps + 1):
        iter_sum = np.zeros(dim)

        # Try except makes sure we don't try to index results too far back. Could be solved with a bool check changed when i >= p
        for j in range(p):
            try:
                iter_sum += coefficients[j] @ results[i - (p + 1)]
            except IndexError:
                break
        
        # Save iteration to results
        iter_sum += np.random.multivariate_normal(mean=np.zeros(dim), cov=noise_cov)
        results[i] = iter_sum
    
    return results
    



#### Kode til at hente data fra nettet
Koden hernede henter data fra FRED. Det er en blanding af lidt økonomisk tidsserie data. Funktionen `get_fred_data()` bruger `pandas-datareader`til at hente det ned.

Funktionen `interpolate_data` tager den data liste der er genereret af `get_fred_data()` og interpolerer det, sådan at vi har et lieg antal punkter for hvert data-sæt. 

In [3]:
def get_fred_data():
    """Pulls data from FRED

    Returns:
        List of arrays: A list of time-series data pulled from the FRED website.
    """
    # Info to get data from fred
    data_names = [
            "PCOPPUSDM",
            "PWHEAMTUSDM",
            "PRAWMINDEXM",
            "APU000072610",
            "APU0000703112",
            "DHHNGSP",
            "PSUNOUSDM",
            "WTISPLC",
            "DCOILWTICO",
            "DCOILBRENTEU",
            "PNGASEUUSDM",
            "APU0000708111",
        ]
    dates = ["1990-01-01", "2020-01-01"]
    data_list = []

    # load the data from the website
    for i in range(len(data_names)):
        name = data_names[i]
        _dat = pdr.get_data_fred(name, dates[0], dates[1])
        _dat = _dat.interpolate(method='nearest').ffill().bfill()  # interpolate to remove NaN values
        data_list.append(_dat)
    
    # Convert to numpy
    arr_dat = []
    for dat in data_list:
        arr_dat.append(dat.to_numpy())
    
    return arr_dat


def interpolate_data(data_list, recalled=False):
    """Interpolates the data loaded by the function get_fred_data(). Should be able to be generalized if need be.

    Args:
        data_list (List of arrays): List returned by get_fred_data()
        recalled (bool, optional): DO NOT CHANGE MANUALLY. This boolean was used when developing the function. If set to true no second run will be done when trying to interpolate. Defaults to False.

    Raises:
        Exception: Raises an exception if NaN is present in the interpolated data. If that is the case Anders will jump in front of a bus.

    Returns:
        List of arrays: List of interpolated time-series data from get_fred_data()
    """
    max_data_len = 0
    min_data_len = np.Inf

    # get longest length of data
    for dat in data_list:
        max_data_len = max(max_data_len, len(dat))
        min_data_len = min(min_data_len, len(dat))
    
    # we interpolate from 0 to 10e4 such that we have an axis, and dont need much decimal precision
    interp_axis = np.linspace(1, 10e4-1, max_data_len)
    interpolated_data = []

    for dat in data_list:
        n = len(dat)
        if n != max_data_len:
            _axis = np.linspace(0, 10e4, n)
            f = interp1d(_axis, dat.reshape(len(dat)), kind='cubic', fill_value=np.mean(dat), bounds_error=False)
            inter_data = f(interp_axis)
        
        else:
            inter_data = dat
        
        interpolated_data.append(inter_data.reshape(len(inter_data)))

    # Scipy is kinda weird, so we check if there are NaN values in the interpolated data
    has_nan = []
    
    # check each array if it contains nan
    for inter in interpolated_data:
        has_nan.append(np.isnan(np.sum(inter)))
    
    if True not in has_nan:  # everything went according to plan, we return the data
        return interpolated_data
    
    if not recalled:  # something went wrong in the first case, we try again
        interpolate_data(data_list, recalled=True)
    
    # We tried twice, and didn't get the data to interpolate
    raise Exception("Data would not interpolate in two tries in function interpolate_data()")

    return interpolated_data


def create_coefficients(data_list, series_list, model_order):
    """Creates the coefficients needed to simulate the data, using statsmodels package. 

    Args:
        data_list (List of arrays): List of time-series data returned by the interpolation function.
        series_list (List of int): Indexes (0, ..., 11) which decides what data to use in the model. Len(series_list) = k -> model of dimension k
        model_order (Int): Order of the model. Must be int >= 1

    Returns:
        Tuple (array, array): Results from VAR.fit(model_order)
            array: coefficient matrices. Size is [p, k, k] where p is the model order, and k is the dimension of the model
            array: covariance matrix. Size is [k, k]
    """
    new_data = []

    for idx in series_list:
        new_data.append(data_list[idx])
    
    exog_var = np.array(new_data)

    model = VAR(exog_var.T)
    result = model.fit(model_order)

    return result.coefs, result.sigma_u


def save_load_coefs(name, coefficients=None, covariance=None, save=False):
    """Save or Load coeffiecients from the AR(p) model

    Args:
        name (str): name under which the coefficients should be save. using name='abc' saves or loads files 'abc_coefs.npy' and 'abc_cov.npy'
        coefficients (array): array of the coefficients returned from create_coefficients()
        covariance (array): array of the covariance returned from create_coefficients()
        save (bool, optional): If set to True the function saves the data specified in coefficients and covariance. If False the functions loads files specified by name. Defaults to False.

    Returns:
        Tuple (array, array | None): If save is false the function returns None
            array: Coefficient matrix loaded by name
            array: Covariance matrix loaded by name
    """
    coef_str = name + "_coefs.npy"
    cov_str = name + "_cov.npy"

    if save:
        np.save(coefficients, coef_str)
        np.save(covariance, cov_str)

        return None
    
    coefs = np.load(coef_str)
    cov = np.load(cov_str)

    return coefs, cov